In [1]:
import numpy as np
import tensorflow as tf 
import tflearn
from collections import deque
import random
import pdb
from replay_memory import ReplayMemory
from AC_networks import ActorNet, CriticNet
from test_game import test_env
import os

In [2]:
class input_struct:
    def __init__(self, net):
        self.net = net


In [3]:
import gym
game = gym.make('LunarLander-v2')
#game = gym.make('CartPole-v0')
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n

[2016-12-28 19:29:58,652] Making new env: LunarLander-v2


In [6]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
MAX_EPISODE = 10000
MAX_STEP = 1000
MINIBATCH_SIZE = 8
GAMMA = 0.99
LEARNING_RATE = 2e-2 #5e-5 lr/2
TAU = 0
ENTROPY_BETA = 0
buff = ReplayMemory(500)
info = input_struct('actor')
try:
    actor
except:
    pass
else:
    actor.reset_net()
try:
    critic
except:
    pass
else:
    critic.reset_net()  
    
graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.act_hid_layers = np.array([16])
    info.crit_hid_layers = np.array([16])
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.ent_beta = ENTROPY_BETA
    actor = ActorNet(info)
    critic = CriticNet(info, actor.get_num_trainable_vars())

In [7]:
#game = test_env()
print_ep = 100
save_ep = 1000
lr_decay_ep = 100
lr = LEARNING_RATE
buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.initialize_all_variables())
    
    #saver = tf.train.Saver()
    #filename = './saved_nets - last/Episod_4999.chk'
    #saver.restore(sess, filename)
    #pdb.set_trace()
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    avg_var = 0
    
    for ep in xrange(MAX_EPISODE):
        s = game.reset()
        var = 0
        #ep_reward[ep] = 0
        #print(ep)
        for st in xrange(MAX_STEP):
            A = actor.take_action(s.reshape(-1,info.dim_s))
            a = np.argmax(A)
            s2, r, terminal, report = game.step(a)
            
            #print(terminal, r)
            #s2, r = game.step(a)
            #terminal = 1
            #print('s: %d,  a: %d,   s2: %d,   r: %d' %(s, np.argmax(A), s2, r))
            
            buff.add(np.reshape(s, (actor.dim_s,)).astype(np.float32), np.reshape(A, (actor.dim_a,)).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (actor.dim_s,)).astype(np.float32),\
                     np.reshape(terminal, (1,)))

            ep_reward[ep] += (1.*r)
            #pdb.set_trace()
            var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2
            #if buff.size() > MINIBATCH_SIZE:
            
            
            
            if terminal:
                var = var/st
                avg_var += (var/print_ep)
                break
                
        if ((ep % MINIBATCH_SIZE) == (MINIBATCH_SIZE -1)):
            (s_batch, a_batch, r_batch, s2_batch, t_batch) = \
                buff.sample_batch(MINIBATCH_SIZE)
            start_update = 1
            s_batch = np.array(s_batch).reshape(-1, info.dim_s)
            s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
            a_batch = np.array(a_batch).reshape(-1, info.dim_a)
            r_batch = np.array(r_batch).reshape(-1, 1)
            t_batch = np.array(t_batch).reshape(-1, 1)

            

            #Q_2 = critic.predict_target(s2_batch)
            Q_2 = critic.predict(s2_batch)

            R_batch = r_batch + GAMMA * Q_2 * (1-t_batch)
            #if(np.max(t_batch) == 1):
            #pdb.set_trace()

            predicted_Q, _ = critic.train(s_batch, np.reshape(R_batch, (-1, 1)), lr/2)
            #predicted_Q, _ = critic.train(s_batch, R_batch, lr/2)

            td_batch = R_batch - predicted_Q
            #pdb.set_trace()
            actor.train(s_batch, a_batch, np.reshape(td_batch, (-1, 1)), lr)
            #pdb.set_trace()
                
        if terminal == False:
            var = var/MAX_STEP
            avg_var += (var/print_ep)
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        #if(start_update):
        #    actor.update_target_network
        #    critic.update_target_network
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Episode: %d --- Ave R: %.1f  |  Success: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2f*1e-4'\
                  %(ep, np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), var, 1e4*lr))
            last_ep = ep
            avg_var = 0
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ******' %ep)
            saver = tf.train.Saver()
            filename = './saved_nets/Episod_' + str(ep) + '.chk'
            try:
                saver.save(sess, filename)
            except:
                os.mkdir('./saved_nets')  
            with open('./saved_nets/rewards_file.txt', 'w') as f:
                for i in xrange(MAX_EPISODE):
                    f.write('%f\n' %ep_reward[i])
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = 1 * lr
            
            
            
    
print('DONE') 
    

Instructions for updating:
Use `tf.global_variables_initializer` instead.


[2016-12-28 19:45:49,262] From <ipython-input-7-95f82d36f5ab>:12 in <module>.: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Episode: 0 --- Ave R: -155.8  |  Success: 0.00  |  crash: 1.00  |  Max R: -155.8  |  var: 0.25  |  lr:200.00*1e-4
Episode: 99 --- Ave R: -220.1  |  Success: 0.00  |  crash: 1.00  |  Max R: -26.0  |  var: 0.26  |  lr:200.00*1e-4
Episode: 199 --- Ave R: -223.6  |  Success: 0.00  |  crash: 1.00  |  Max R: -65.0  |  var: 0.27  |  lr:194.00*1e-4
Episode: 299 --- Ave R: -252.9  |  Success: 0.00  |  crash: 0.99  |  Max R: -6.9  |  var: 0.30  |  lr:188.18*1e-4
Episode: 399 --- Ave R: -363.3  |  Success: 0.00  |  crash: 1.00  |  Max R: -65.9  |  var: 0.42  |  lr:182.53*1e-4
Episode: 499 --- Ave R: -407.0  |  Success: 0.00  |  crash: 1.00  |  Max R: -3.4  |  var: 0.40  |  lr:177.06*1e-4
Episode: 599 --- Ave R: -402.8  |  Success: 0.00  |  crash: 1.00  |  Max R: -105.1  |  var: 0.32  |  lr:171.75*1e-4
Episode: 699 --- Ave R: -358.0  |  Success: 0.00  |  crash: 1.00  |  Max R: -43.2  |  var: 0.32  |  lr:166.59*1e-4
Episode: 799 --- Ave R: -224.8  |  Success: 0.00  |  crash: 0.99  |  Max R: -13.9  

[2016-12-28 19:47:20,427] Error encountered when serializing data_augmentation.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


...

Episode: 9099 --- Ave R: -95.3  |  Success: 0.00  |  crash: 1.00  |  Max R: -11.8  |  var: 0.34  |  lr:12.90*1e-4
Episode: 9199 --- Ave R: -101.6  |  Success: 0.00  |  crash: 1.00  |  Max R: -14.9  |  var: 0.33  |  lr:12.51*1e-4
Episode: 9299 --- Ave R: -121.0  |  Success: 0.00  |  crash: 0.99  |  Max R: 4.0  |  var: 0.35  |  lr:12.13*1e-4
Episode: 9399 --- Ave R: -101.0  |  Success: 0.00  |  crash: 0.99  |  Max R: 7.2  |  var: 0.36  |  lr:11.77*1e-4
Episode: 9499 --- Ave R: -107.8  |  Success: 0.00  |  crash: 0.99  |  Max R: -20.1  |  var: 0.36  |  lr:11.42*1e-4
Episode: 9599 --- Ave R: -102.9  |  Success: 0.00  |  crash: 0.97  |  Max R: 8.0  |  var: 0.34  |  lr:11.08*1e-4
Episode: 9699 --- Ave R: -121.0  |  Success: 0.00  |  crash: 0.99  |  Max R: -20.2  |  var: 0.37  |  lr:10.74*1e-4
Episode: 9799 --- Ave R: -119.0  |  Success: 0.00  |  crash: 1.00  |  Max R: -5.8  |  var: 0.35  |  lr:10.42*1e-4
Episode: 9899 --- Ave R: -116.6  |  Success: 0.00  |  crash: 0.98  |  Max R: -17.3  |  v

[2016-12-28 20:08:50,358] Error encountered when serializing data_augmentation.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


[2016-12-28 20:08:50,363] Error encountered when serializing data_preprocessing.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


DONE


In [ ]:
EP = MAX_EPISODE - 1
with tf.Session(graph=graph) as sess:
    
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.initialize_all_variables())
    saver = tf.train.Saver()
    filename = './saved_nets/Episod_' + str(EP) + '.chk'
    saver.restore(sess, filename)
    
    ep_reward = 0
    s = game.reset()
    for st in xrange(MAX_STEP):
        game.render()
        print(actor.give_policy(s.reshape(-1,info.dim_s)))
        A = actor.take_action(s.reshape(-1,info.dim_s))
        a = np.argmax(A)
        s2, r, terminal, report = game.step(a)
        s = s2
        ep_reward += (1.*r)
        if terminal:
            break
    print(r)
    print('reward:%f' %np.mean(ep_reward))

In [ ]:
with open('rewards_file.txt', 'w') as f:
    for i in xrange(MAX_EPISODE):
        f.write('%f\n' %ep_reward[i])


## 